In [ ]:
basedir = '/home/abhinavgupta0110/NeuralODEs_ROM_Closure'

import os

is_google_colab = False
is_use_GPU = False

### Mount the Google drive if needed

In [ ]:
if is_use_GPU:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('No GPU found!')
    else:
        print(gpu_info)

if is_google_colab:
    from google.colab import drive
    drive.mount('/content/drive')

    %pip install quadpy
    
os.chdir(os.path.join(basedir, 'neuralClosureModels'))

%load_ext autoreload

### Load modules

In [ ]:
%autoreload 2

from src.utilities.DDE_Solver import ddeinttf 
from src.utilities.helper_classes import * 
import src.solvers.neuralDDE_with_adjoint_accel as ndde
import src.advec_diff_case.advec_diff_eqn_modcall as adeq
from src.advec_diff_case.AD_Eqn_Helper_Classes import * 

import time
import sys
from IPython.core.debugger import set_trace

import numpy as np
import scipy as spy
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from shutil import move
import pickle

print(tf.__version__)

tf.keras.backend.set_floatx('float32')
import logging
tf.get_logger().setLevel(logging.ERROR)

## Define some useful classes

### Class for user-defined arguments

In [ ]:
class res_ad_eq_nDDE_args(ndde.arguments, adeq.ad_eqn_args):

    def __init__(self, batch_time = 12, batch_time_skip = 2, batch_size = 5, epochs = 500, learning_rate = 0.05, decay_rate = 0.95, test_freq = 1, plot_freq = 2, 
                 d_max = 1.1, rnn_nmax = 3, rnn_dt = 0.5, state_dim = 2, adj_data_size = 2,
                 model_dir = 'ROM_nODE_testcase/model_dir_test', restart = 0, val_percentage = 0.2,
                 T = 2., nt = 200, L = 1., nx_high_res = 100, nx_low_res = 25, Re = 250, u_bc_0 = 0., u_bc_L =0., isplot = True, is_tstart_zero = True, 
                 ode_alg_name = 'dopri5', nsteps = 1): # add more arguments as needed
        
        ndde.arguments.__init__(self, data_size = nt, batch_time = batch_time, batch_time_skip = batch_time_skip, batch_size = batch_size, epochs = epochs,
                           learning_rate = learning_rate, decay_rate = decay_rate, test_freq = test_freq, plot_freq = plot_freq, d_max = d_max, rnn_nmax = rnn_nmax, 
                           rnn_dt = rnn_dt, state_dim = nx_low_res, adj_data_size = nx_low_res, model_dir = model_dir, restart = restart, val_percentage = val_percentage, 
                           isplot = isplot, is_tstart_zero = is_tstart_zero)

        adeq.ad_eqn_args.__init__(self, T = T, nt = nt, L = L, nx = nx_low_res, Re = Re, u_bc_0 = u_bc_0, u_bc_L = u_bc_L,
                              multi_solve_size = batch_size)
        
        self.args_for_high_res = adeq.ad_eqn_args(T = T, nt = nt, L = L, nx = nx_high_res, Re = Re, u_bc_0 = u_bc_0, u_bc_L = u_bc_L, 
                              multi_solve_size = 1)
        
        self.args_for_low_res = adeq.ad_eqn_args(T = T, nt = nt, L = L, nx = nx_low_res, Re = Re, u_bc_0 = u_bc_0, u_bc_L = u_bc_L, 
                              multi_solve_size = 1)
        
        self.nx_high_res = nx_high_res
        self.nx_low_res = nx_low_res
        self.ode_alg_name = ode_alg_name
        self.nsteps = nsteps

### Define the neural net architecture

In [ ]:
class DDEFuncMain(tf.keras.Model):

    def __init__(self, args, **kwargs):
        super(DDEFuncMain, self).__init__(**kwargs)

        self.c1 = tf.keras.layers.Conv1D(filters=4, kernel_size=3, strides=1, activation='swish',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.c2 = tf.keras.layers.Conv1D(filters=5, kernel_size=3, strides=1, activation='swish',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.c3 = tf.keras.layers.Conv1D(filters=5, kernel_size=3, strides=1, activation='swish',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.c4 = tf.keras.layers.Conv1D(filters=5, kernel_size=3, strides=1, activation='swish',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.c5 = tf.keras.layers.Conv1D(filters=5, kernel_size=3, strides=1, activation='swish',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.dc1 = tf.keras.layers.Conv1DTranspose(filters=3, kernel_size=3, strides=1, activation='swish',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.dc2 = tf.keras.layers.Conv1DTranspose(filters=2, kernel_size=3, strides=1, activation='swish',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.dc3 = tf.keras.layers.Conv1DTranspose(filters=2, kernel_size=3, strides=1, activation='swish',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.dc4 = tf.keras.layers.Conv1DTranspose(filters=2, kernel_size=3, strides=1, activation='swish',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.dc_out = tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=3, strides=1, activation='linear',
                                       kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1), use_bias=True)
        
        self.flat = tf.keras.layers.Flatten()
        
        self.args = args


    @tf.function
    def pass_layers(self, y_nn):
        
        for i in range(len(self.layers)):
            y_nn = self.layers[i](y_nn)
            
        return y_nn
    
    def call(self, y_nn):
        
        y_nn = self.pass_layers(y_nn)
        
        return y_nn

In [ ]:
class DiscDDEFunc(tf.keras.Model):

    def __init__(self, main, rom_model, args, **kwargs):
        super(DiscDDEFunc, self).__init__(**kwargs)
        
        self.main = main
        self.rom_model = rom_model
        self.args = args
        
    def process_input(self, y, t ,d):     
        
        input = tf.expand_dims(y(t), axis=-1)
        
        return input

    def call_nn_part(self, input):
        dy_dt = self.main(input)
        return dy_dt
    
    def __call__(self, y, t ,d, t_start = np.array([0.])):
        
        y_nn = self.process_input(y, t, d)

        dy_dt = self.call_nn_part(y_nn) + self.rom_model(y, t, t_start)
        
        return dy_dt

### Initialize model related parameters

In [ ]:
args = res_ad_eq_nDDE_args(batch_time = 6, batch_time_skip = 2, batch_size = 8, epochs = 250, learning_rate = 0.075, 
                           decay_rate = 0.97, test_freq = 1, plot_freq = 2, d_max = 0.2, rnn_nmax = 1, rnn_dt = 0.1, 
                           model_dir = 'Res_nODE_testcase_v2/model_dir_case_test', restart = 0, val_percentage = 1.,
                           T = 1.25, nt = 125, L = 1., nx_high_res = 100, nx_low_res = 25, Re = 1000, u_bc_0 = 0., 
                           u_bc_L =0., ode_alg_name = 'vode', nsteps = -1)

### Make a copy of the current script

In [ ]:
testcase_dir = 'neuralClosureModels/testcases/AD_Eqn_Res'
save_dir_obj = save_dir(args = args, basedir = basedir, testcase_dir = testcase_dir)
save_dir_obj(script_name = 'neuralODE_Res_AD_Eqn_TestCase-Conv1D-Accel')

### Run setup

In [ ]:
os.chdir(os.path.join(basedir, testcase_dir))

%run -i setup

## Main part starts here

### Make objects and define learning-rate schedule

In [ ]:
time_meter = ndde.RunningAverageMeter(0.97)

adv_eq = adeq.ad_eqn(op = adeq.operators(args), app = args)
func_main = DDEFuncMain(args)
func = DiscDDEFunc(func_main, adv_eq, args)
adj_func = ndde.adj_eqn(func, args, adv_eq.jac)
get_batch = ndde.create_batch(true_u_low_res, u0, t, args)
loss_obj = custom_loss()
plot_obj = custom_plot(tf.concat([true_u_low_res, val_true_u_low_res], axis=0), 
                       tf.concat([u_low_res, val_u_low_res], axis=0), x_low_res, tf.concat([t, val_t], axis=0), save_dir_obj.figsave_dir, args)
loss_history = ndde.history(args)
val_obj = ndde.create_validation_set(u0, t, args)

initial_learning_rate = args.learning_rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=args.niters,
    decay_rate=args.decay_rate,
    staircase=True)

### Quick test to see how the true coefficients looks like

In [ ]:
if args.restart == 1: 
    func.load_weights(tf.train.latest_checkpoint(save_dir_obj.checkpoint_dir))
    pred_y = ddeinttf(func, u0, tf.concat([t, val_t], axis=0), fargs=([args.rnn_nmax, args.rnn_dt],), alg_name = args.ode_alg_name, nsteps = args.nsteps)
    
    plot_obj.plot(pred_y, epoch = 0)

    loss_history.read()
    
    initial_learning_rate = 0.05
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=args.niters, decay_rate=0.96, staircase=True)
    
else:
    plot_obj.plot(epoch = 0)

### Training starts here

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr_schedule)

nDDE_train_obj = ndde.train_nDDE(func = func, adj_func = adj_func, d = [args.rnn_nmax, args.rnn_dt], loss_obj = loss_obj, batch_obj = get_batch,
                            optimizer = optimizer, args = args, plot_obj = plot_obj, time_meter = time_meter, checkpoint_dir = save_dir_obj.checkpoint_prefix, 
                            validation_obj = val_obj, loss_history_obj = loss_history)

nDDE_train_obj.train(true_u_low_res, u0, t, val_true_u_low_res)